# Tarea 2 IA

> Mauricio Montanares

> Sebastian Mendoza

Este dataset contiene datos de una planta procesadora de minerales. 

En la industria minera es de vital importancia conocer el nivel de impurezas de la pulpa final de la etapa de flotación. Los métodos tradicionales consisten en realizar pruebas químicas en base a muestras de está pulpa al final del proceso de flotación, no hay métodos que permitan una estimación previa de los niveles de contaminantes.

Conocer el % de contaminación de la pulpa puede ayudar a reducir pérdidas en la calidad del producto final y posibles pérdidas de dinero.

En este trabajo se implementan modelos Min Max Scaler, Random Forest Regressor, Support Vector Machines, Classic NN y LSTM Implementation.

Se recalca que este es un problema de regresión, ya que se busca predecir valores continuos, y no clasificar.



In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
#data_clean = pd.read_csv("drive/My Drive/Colab Notebooks/CpcProyect/data/data_cleaned.csv")
data_mining = pd.read_csv('MiningProcess_Flotation_Plant_Database.csv',decimal=",",parse_dates=["date"],infer_datetime_format=True).drop_duplicates()
#data_mining = pd.read_csv('drive/My Drive/Colab Notebooks/CpcProyect/data/Copia de MiningProcess_Flotation_Plant_Database.csv')

In [ ]:
data_mining.head().transpose() #transpose the data

In [ ]:
#change the order of columns (the last column) for do the job more easy

data_mining = data_mining[['date', '% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level', 
        '% Silica Concentrate','% Iron Concentrate']]

# First view of the data

In [ ]:
data_mining.shape

In [ ]:
#create corrmatrix
plt.figure(figsize=(30, 25))
corr_matrix = data_mining.corr()

sn.heatmap(corr_matrix , annot=True)
plt.savefig("corr_matrix.eps")
plt.show()

> No observamos ninguna correlacion evidente ademas de la correlacion inversa entre la concentracion de silice y la de hierro. Esto ultimo coincide con la relacion inversamente proporcional de la cual la fuente del dataset hacia mencion.

In [ ]:
df_corr = pd.DataFrame(corr_matrix)


corr_values = np.array(df_corr.iloc[22]) #drop IRON
corr_values = corr_values[0:len(corr_values)-1]

corr_names = np.array(df_corr.columns)
corr_names = corr_names[0:len(corr_names)-1] #drop iron
#corr_names = corr_names + corr_names[len(corr_names)] #add silica 

plt.figure(figsize=(55,25))
plt.barh(corr_names, corr_values, color="salmon")
plt.title("Correlation Values for % Iron Concentrate", fontsize=45)
plt.xticks(fontsize=30,rotation=90)
plt.yticks(fontsize=30)
#plt.savefig("corr_silicia.eps")
#plt.savefig("corr_silicia.svg")
plt.show()

# Resampling.

> La fuente del dataset especifica que existen variables medidas cada 1 minuto y otras (silica y concentracion de hierro) que son medidas cada 1 hora aproximadamente.

> Nuestro resampling considera un promedio de las muestras tomadas cada 1 minuto para generar un nuevo dato, este dato es almacenado en nuevo dataset.

In [ ]:
#resampling with mean 

names = ['% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow',
       'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density',
       'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
       'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
       'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
       'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
       'Flotation Column 02 Level', 'Flotation Column 03 Level',
       'Flotation Column 04 Level', 'Flotation Column 05 Level',
       'Flotation Column 06 Level', 'Flotation Column 07 Level',
       '% Iron Concentrate', '% Silica Concentrate']
 
 
ts = 175
for x in range(1,data_mining.shape[1]):
  data = [] #clear data 
  n = 0 #set counter
 
  while n*ts <= round(data_mining.shape[0]):
    sample_mean_value = data_mining.iloc[(ts*n):(ts*(n+1)),x].mean()  #sampling data and save mean of n*ts elements
    data += [sample_mean_value]   #save sampling into a list
    data_array = np.array(data)   #list to array
    data_array= np.transpose(data_array)  #transpose
    df = pd.DataFrame(data_array)   #array to dataframe
    n += 1  #counter up 
 
  if x == 1:
    df_first = df
  if x == 2:
    new_df = pd.concat([df_first, df], axis = 1)
  if x > 2: 
    new_df = pd.concat([new_df, df], axis=1) 
 
new_df.columns = names

In [ ]:
new_df.to_csv('resampling_data.csv')

> *We will work with this new dataset!*

In [ ]:
#remove output %Silica concentrate for post ML analysis 
new_df = new_df.drop(labels=[ '% Silica Concentrate'], axis = 1)

# MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
names = new_df.columns #store the names

In [ ]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(new_df)
scaled_df = pd.DataFrame(scaled_df, columns=names)

In [ ]:
scaled_df

> Now data it's normalized

# Random Forest Regressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
data_mining = scaled_df  #change the name... 
X = data_mining.drop(['% Iron Concentrate'], axis=1) #drop target 
y = data_mining['% Iron Concentrate']  #select target 

#make split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
# Set the parameters by cross-validation and GridSearch
para_grids = {
            "n_estimators" : [10,50,100,200],
            "max_features" : ["auto", "log2", "sqrt"],
            "bootstrap"    : [True, False]
        }

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    estimator = RandomForestRegressor()
    clf = GridSearchCV(estimator, para_grids)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)




In [ ]:
model = RandomForestRegressor(n_estimators=200, max_features='sqrt', bootstrap = False) #utilizamos los parametros anteriors

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test,y_test)

> SCORE 0.48

In [ ]:
#PLOT PREDICT VALUES VS REAL VALUES
pred = model.predict(X_test)
plt.figure(figsize=(20, 10))
plt.plot(y_test.values[1:600]) #JUST SOME DATA
plt.plot(pred[1:600])
plt.savefig("regressionRF.pdf")
plt.show()

# Support Vector Machines

In [ ]:
from sklearn.svm import SVR

In [ ]:
#WARNING: be careful with this code. Take  A LOT OF TIME in finish!
#param = { ,

#model = SVR()

#grids = grid_search = GridSearchCV(estimator = model, param_grid = param, 
                    #  cv = 3, n_jobs = 4, verbose = 2)

#grids.fit(X_train, y_train)
# Tuning hyper-parameters for precision


#print("Best parameters set found on development set:")
#print()
#print(grid_search.best_params_)



In [ ]:
regressor = SVR(kernel = 'poly', C = 50, degree=8, coef0=0.5, gamma='auto')
regressor.fit(X_train, y_train)
regressor.score(X_test,y_test)

> SCORE 0.3217

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(20, 10))
plt.plot(y_test.values[1:600])
plt.plot(pred[1:600])
plt.savefig("regressionSVRF.pdf")
plt.show()

# CLASSIC NN

In [ ]:
#define x's and y 
x = data_mining.drop(labels='% Iron Concentrate', axis=1)
y = data_mining['% Iron Concentrate']

In [ ]:
# Create train/test
from sklearn.model_selection import train_test_split
 
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.3)

In [ ]:
#Import TensorFlow and keras
 
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import zscore

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import zscore

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)

In [ ]:
 
#Build a NN
 
model = Sequential()
model.add(Dense(22, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(18, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train,y_train,batch_size = 10,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=100)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

> Not so good...

In [ ]:
# summarize history for loss
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

> Se observa una disminucion pero no es rapida ni constante

In [ ]:
pred = model.predict(x_test)
plt.figure(figsize=(20, 10))
plt.plot(y_test.values[1:600])
plt.plot(pred[1:600])
plt.savefig("regression.pdf")
plt.show()

In [ ]:
from sklearn import metrics
 
# Predict
pred = model.predict(x_test)
 
# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

In [ ]:
 
import numpy as np
 
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))



# LSTM IMPLEMENTATION with mean()

> Al ser un dataset del tipo temporal(dependencia temporal) los datos en t, t+1, dependen de t-n

> Existen arquitecturas especiales para estos tipos de problemas, una de ellas es la LSTM (Long short-term memory)



In [ ]:

from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 
# convert series to supervised learning 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg



values = new_df.values

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[(reframed.shape[1]-21):(reframed.shape[1]-1)], axis=1, inplace=True)

# split into train and test sets
values = reframed.values
n_train_hours = round(reframed.shape[0] * 0.8) #70% of data for training
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


from tensorflow.keras.callbacks import EarlyStopping



# design network
model = Sequential()
model.add(LSTM(5, activation='sigmoid', input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=20, validation_data=(test_X, test_y), verbose=2, shuffle=False)

 




# Test Model and some plots 

In [ ]:
# plot history
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend(fontsize=15)
plt.xlabel('epochs', fontsize = 15)
plt.ylabel('Loss value', fontsize = 15)



plt.savefig("lossLSTM.svg")
plt.show()

In [ ]:
pred = model.predict(test_X)
plt.figure(figsize=(15, 8))

#subplots

plt.subplot(211)


plt.plot(pred[1:int(818/2),0], label='predict', color='black')
plt.plot(test_y[:int(818/2)], label='test', color = 'salmon')
plt.title('LSTM. Predicted values and test values', fontsize=20)
plt.ylabel('%Iron Concentrate', fontsize = 15)
plt.legend(fontsize = 15)


plt.subplot(212)
plt.plot(np.arange(int(820/2),841),pred[int(820/2):,0], label='predict', color='black')
plt.plot(np.arange(int(820/2),842),test_y[int(818/2):], label='test', color = 'salmon')


plt.xlabel('Data[Hours]', fontsize = 15)
plt.ylabel('%Iron Concentrate', fontsize = 15)
plt.legend(fontsize = 15)

plt.savefig("regressionLSTM.svg")

plt.show()

In [ ]:
#error plot 

pred = model.predict(test_X)
plt.figure(figsize=(11, 8))

error = (test_y[:818]-pred[1:819,0]) * 100

#error_percent = error*100/test_y.max()

plt.plot(range(error.shape[0]) ,error, color = 'red')

plt.title('LSTM. Error predicted values vs test values', fontsize=20)
plt.xlabel('Data[Hours]', fontsize = 15)
plt.ylabel('%Error', fontsize = 15)
#plt.legend(fontsize = 15)

plt.grid(b=True, which='major', color='#999999', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.savefig("LSTM_error.pdf")
plt.show()

In [ ]:
error.max()

# METRICS LSTM

In [ ]:
from sklearn import metrics

# Predict
pred = model.predict(test_X)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,test_y)
print("Final score (MSE): {}".format(score))

In [ ]:
import numpy as np

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_y))
print("Final score (RMSE): {}".format(score))

In [ ]:
#mae
from sklearn.metrics import mean_absolute_error
pred = model.predict(test_X)
score = mean_absolute_error(test_y, pred, multioutput='raw_values')
print("Final score (MAE): {}".format(score))